In [1]:
from scraper import Corpus
from data_preparation import DataPreparation
from analyzer import Analyzer

In [ ]:
%load_ext autoreload
%autoreload 2

# TODO



# WORKFLOW

1. Create Corpus with `Corpus.initCorpus()` (once initialized, you do not have to initialize it again).
2. Prepare the website data in the corpus (via `DataPreparation()` class).

    1. Download and analyze images and videos via `DataPreparation.getImages()`.
    2. Once you have downloaded the data for the current corpus, you do not need to download and analyze the images and videos again. You can now access the collected data via `DataPreparation.getImagesFromPickle()`.
    3. Create and save a dictionary with all the results via `DataPreparation.createAnalyzerDict()`. You need to crate this dictionary before you can proceed with the analysis.

3. The clustering of the data is conducted with the `Analyzer` class.
    
    1.To cluster the data in the dictionary created with `DataPreparation.createAnalyzerDict()`, the user can choose the columns that should be selected with the help of the method `Analyzer.setColumnSelection()`. The data can then be clustered with the method `Analyzer.clusterDataKMeans()`. However, it is advisable to standardize the data first with the method `Analyzer.standardizeData()`.
    2. To figure out which n works best in the `Analyzer.clusterDataKMeans()` method, the user can use `Analyzer.createElbowPlot()`.
    3. The results of the clustering can be visualized with `Analyzer.visualizeCluster()`-in this case, the user needs to choose two features first.

# Corpus Creation

In [ ]:
cp = Corpus()

In [ ]:
cp.initCorpus()

# Data Preparation

In [ ]:
dp = DataPreparation()

In [ ]:
#dp.getImages()

In [ ]:
data_dict = dp.createAnalyzerDict()

# Analysis

In [ ]:
ana = Analyzer()

In [ ]:
ana.getColumnSelection()

In [ ]:
df = ana.getFullDataFrame()

In [ ]:
df["domain"]

In [ ]:
# KMeans clustering and standardization

In [ ]:
ana.setColumnSelection(['RA_big_images/total_images', 'total_videos'])

In [ ]:
ana_selection = ana.getDataFrame()

In [ ]:
standardized_data = ana.standardizeData(ana_selection)

In [ ]:
standardized_data

In [ ]:
clustered_data = ana.clusterDataKMeans(standardized_data, n=3)

In [ ]:
ana.createElbowPlot(clustered_data)

In [ ]:
#ana.visualizeCluster(clustered_data, "total_length", "total_images")

In [ ]:
ana.getScreenshotsFromClusters(clustered_data)

In [ ]:
clustered_data

In [ ]:
ana.data_df

# Testing stuff